# Imports classiques
Nous allons tout d'abord lancer les imports classiques

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import functions as func
import Optim_corr as opt
import functions_corr as func

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'Optim_corr'

## Modification des algorithmes
Nous allons modifier la structure des algorithmes d'optimisation et les données qu'ils rendent. Au lieu de rendre plusieurs listes `x_list,cost_list,grad_list,...`, les algorithmes vont rendre un seul résultat `to_return` qui est un dictionnaire qui aura plusieurs entrées. Voici une liste de commandes pour comprendre le fonctionnement d'un dictionnaire 

Copiez votre fichier dans un fichier `Optim.py` dans un fichier `Optim2.py`. Et modifiez toutes vos fonctions pour quelles vous rendent juste un dictionnaire.
Nous allons maintenant créer une fonction `Wolfe(J,x,d,cost,grad,s=1.,itermax=20,e1=1.e-4,e2=0.99)` qui remplace la recherche de pas de Wolfe si on va dans la direction de descente `d`. les variables `cost` et `grad` sont respectivement la valeur de `J(x)` et le gradient $\nabla J(x)$ Je l'ai mise dans un fichier `LibOptim.py`. Cette fonction devra vous rendre `s,cost_new,grad_new` qui sont respectivement : le pas trouvé par la méthode de Wolfe, le coût au nouveau point et le gradient au nouveau point, c'est à dire, respectivement $J(x+sd)$ et $\nabla J(x+sd)$.
Modifiez en conséquence votre fonction `Optim2.py` et vérifiez que vous n'avez rien cassé en lançant le genre de commandes suivantes :

In [7]:
import LibOptim as lib
import Optim as opt1
import Optim2 as opt

f=func.Rosen()
x0=np.array([-1,0])
x_list,cost_list,grad_list,step_list = opt1.wolfe(f,x0,itermax=10000)
r = opt.wolfe(f,x0,itermax=10000)
diff = np.array(cost_list)-np.array(r['cost_list'])
print(diff.shape, np.linalg.norm(diff))

(10000,) 0.0


## Méthode de Newton
Nous nous intéressons à la méthode de Newton, il s'agit ici de choisir une direction de descente
$$d_k=H[J](v_k)^{-1}\nabla J(v_k)$$
et un pas de $1$
Créez une fonction `Newton(J,x_0,itermax=500,tol=1.e-5)` dans le fichier `Optim2.py` et lancez la méthode de Newton sur les différentes fonctions tests. Cette méthode vous un dictionnaire avec les entrées `x_list,cost_list,grad_list,step_list`, voir le TP1 pour les explications sur les différentes variables.
Vous rajouterez aussi dans le dictionnaire l'entrée `angle_list` qui est le cosinus de l'angle fait entre la direction de descente choisie par l'algorithme et le gradient. On vérifiera que `angle_list` est négatif et proche de `1`. 
Pour faire l'inversion du système, on utilisera la fonction `solve` de la bibliothèque `scipy.linalg`.
Testez ci-dessous votre algorithme pour les différentes fonctions.

In [9]:
f = func.Rosen()
x0 = np.array([-1, 0])
r = opt.Newton(f, x0, itermax=500, tol=1.e-5)
print(r)
print(f.fcounter)
print(f.gcounter)
print(f.hcounter)

{'x_list': [array([-1,  0]), array([-0.99004975,  0.9800995 ]), array([ 0.9613592 , -2.88378537]), array([0.96140987, 0.92430893]), array([0.99999998, 0.99851076]), array([1.        , 0.99999999]), array([1., 1.])], 'cost_list': [3.960298992672956, 1450.0855138980314, 0.0014891985191022043, 0.0002217707674237523, 2.06791962366025e-17, 0.0], 'grad_list': [450.7948535642349, 4.019357199736802, 1650.4854469527404, 0.0771792826529715, 0.6659890941821408, 9.094788033900982e-09], 'step_list': [1, 1, 1, 1, 1, 1], 'angle_list': [0.8992537197422174, 0.9020051331828127, 0.8954177090313139, 0.8954268869086202, 0.9016595473673641, 0.9016537095157616]}
6
6
6


In [4]:
r["x_list"]

[array([-1,  0]),
 array([-0.99004975,  0.9800995 ]),
 array([ 0.9613592 , -2.88378537]),
 array([0.96140987, 0.92430893]),
 array([0.99999998, 0.99851076]),
 array([1.        , 0.99999999]),
 array([1., 1.])]

## Stabilisation de Newton
Normalement vous avez vu que l'algorithme de Newton ne fonctionne pas très bien avec la fonction `oscill`. L'objectif est de stabiliser cet algorithme de Newton en y rajoutant une recherche de pas de style `Wolfe`. Implémentez une foncion `Newton_Wolfe` qui concatène les algorithmes de `Newton` et de `Wolfe`. Vous prendrez bien garde à regarder le nombre de calculs que vous faites de la Hessienne, du gradient et d'évaluations de la fonction.
De plus l'algorithme de Wolfe ne va pas très bien fonctionné si la direction de choisie n'est pas de descente, pour cela si vous trouvez que $(d,\nabla J(x)) >0$, vous prendrez comme direction de descente la direction donnée par $d=-\nabla J(x)$

In [5]:
f = func.oscill()
x0 = np.array([-2, -2])
r = opt.Newton_Wolfe(f, x0, itermax=500, tol=1.e-5)
#print(r)
print(f.fcounter)
print(f.gcounter)
print(f.hcounter)

519
507
165


In [10]:
r

{'x_list': [array([-2., -2.]),
  array([-2.00012207, -2.00012207]),
  array([-2.00011782, -2.0002277 ]),
  array([-2.00026531, -2.00033869]),
  array([-2.0003755 , -2.00049965]),
  array([-2.0003755 , -2.00049965]),
  array([nan, nan]),
  array([nan, nan])],
 'cost_list': [array([2.83281062, 2.83281062]),
  array([2.83299244, 2.83325796]),
  array([2.83355071, 2.83372805]),
  array([2.83410972, 2.83440979]),
  array([2.83410972, 2.83440979]),
  array([nan, nan]),
  array([nan, nan])],
 'grad_list': [3.0240788312284432,
  3.0242706872545706,
  3.024288814976173,
  3.0245123401739584,
  3.0246969139940965,
  3.0246969139940965,
  nan],
 'step_list': [6.103515625e-05,
  6.103515625e-05,
  6.103515625e-05,
  6.103515625e-05,
  4.81482486096809e-35,
  4.81482486096809e-35,
  4.81482486096809e-35],
 'angle_list': [0.5485304671885517,
  0.8425915142414898,
  0.5403023058681395,
  0.5836937224051124,
  0.6513291448813762,
  nan,
  nan]}

## L-BFGS
Nous allons nous intéresser à l'algorithme Limited Memory BFGS. Cet algorithme est du type BFGS, c'est à dire qu'il estime l'inverse de la Hessienne de `J`. Le `L` dans l'algorithme signifie qu'il est à mémoire limitée, c'est à dire qu'il ne garde en mémoire que les $L$ dernières itérations de calcul pour estimer la Hessienne.
L'algorithme est le suivant : Nous sommes à l'itération $k$, nous notons $x_k$ l'itéré et nous avons stocké les vecteurs suivants pour tout $k_{min}\le i\le k$.
$$ s_i=x_{i}-x_{i-1} \text{ et } y_i=\nabla J(x_{i}) -\nabla J(x_{i-1})$$
Et on a aussi stocké $\rho_i=\frac{1}{(s_i,y_i)}$. Tous les $\rho_i$ doivent être positifs.
L'algorithme est le suivant 
# Algorithme

$r=-\nabla J(x_k)$

Pour $i=k,k-1,\dots k_{min}$

... $\alpha_i=\rho_i(s_i \cdot r)$

... $r=r-\alpha_i y_i$

$r=\displaystyle \frac{(s_k\cdot y_k)}{(y_k\cdot y_k)}*r$

Pour $i=k_{min},k_{min}+1,\dots,k$

... $\beta_i=\rho_i(y_i\cdot r)$

... $r=r+(\alpha_i-\beta_i)s_i$

rend r
# Fonctions sur les listes
Vous aurez sans doute besoin des fonctions suivantes pour les listes

In [7]:
a=[(2*i,3*i) for i in range(5) ]
print(a)
print(a[3],a[-1])
print('*** Pop ***')
a.pop(0)
print(a)
print('*** Parcours ***')
for e,f in a :
      print(e,'et',f)
print('*** Parcours Inverse***')
for e in reversed(a) :
      print(e)
b=[e**2 for e,f in a]
print(b)
print('*** Parcours de deux listes ensembles***')
for (m,(t,p)) in zip(b,a) :
    print(m,'et',t,'et encore',p)
print('*** Append ***')
print(b)
b.append(546)
print(b)
print('*** Inversion ***')
c=list(reversed(b))
print(c)
print(len(b))

[(0, 0), (2, 3), (4, 6), (6, 9), (8, 12)]
(6, 9) (8, 12)
*** Pop ***
[(2, 3), (4, 6), (6, 9), (8, 12)]
*** Parcours ***
2 et 3
4 et 6
6 et 9
8 et 12
*** Parcours Inverse***
(8, 12)
(6, 9)
(4, 6)
(2, 3)
[4, 16, 36, 64]
*** Parcours de deux listes ensembles***
4 et 2 et encore 3
16 et 4 et encore 6
36 et 6 et encore 9
64 et 8 et encore 12
*** Append ***
[4, 16, 36, 64]
[4, 16, 36, 64, 546]
*** Inversion ***
[546, 64, 36, 16, 4]
5


# Class BFGS
Créez une classe `BFGS` dans `LibOptim.py`, sa fonction `__init__` sera de la forme 
`__init__(self,nb_stock_max=8)` où `nb_stock_max` est le nombre maximum d'itérations prises en compte. Cette fonction créera aussi une liste vide appelée `stock` qui conserve les $s_i,g_i,\rho_i$. Elle dera aussi créer une liste vide nommée `last_iter`.

# Push
Nous allons maintenant créer une fonction `push(self, x, grad)` qui enregistre $s_k,g_k,\rho_k$. Pour cela, on a besoin de $x_{k-1},\nabla J(x_{k-1})$. Si ils existent, ils se trouvent dans la liste `self.last_iter`. Ensuite on peut calculer $s_k,g_k$ et $\rho_k$. 

Si $\rho_k$ est positif, alors on enregistre le triplet $(s_k,g_k,\rho_k)$ à la fin de la liste `self.stock`, en vérifiant que stock ne doit contenir au maximum que les dernières `self.nb_stock_max` itérations. 

Si $\rho_k$ est négatif, quelquechose c'est mal passé, on vide le `self.stock`.

A la fin, on n'oublie pas de mettre $x_{k}$ et $\nabla J(x_{k})$ dans `self.last_iter` pour être sûr de les y trouver la prochaine fois.

# Get

Nous allons maintenant créer une fonction `get(self, grad)` qui modifie la direction de descente et applique l'algorithme ci-dessus. Cette fonction doit nous rendre le `r` final. Si le `self.stock` est vide, cette fonction doit nous rendre `-grad`

# C'est l'heure de tester ...
Lancez une méthode de Newton_Wolfe sur votre fonction préférée et à chaque itération calculez ce que donnerait un L-BFGS. Comparez les angles des directions entre la méthode de Newton et le L-BFGS, comparez aussi le ration des normes. Ensuite lancez un LBFGS avec recherche de pas de Wolfe sur vos tests préférez et obtenez le comportement de Newton_Wolfe sans le calcul de la Hessienne...


In [8]:
f = func.oscill()
x0 = np.array([-2., -2.])
r = lib.optim_bfgs(f, x0, itermax=500, tol=1.e-5)
#print(r)
print(f.fcounter)
print(f.gcounter)
print(f.hcounter)

warning
warning
476
134
0


/Users/julienblanchon/Git/julien-blanchon/Le-TP-de-Keunotor3/optimisation/tp/tp2/LibOptim.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0/(np.dot(sk, yk))
/Users/julienblanchon/Git/julien-blanchon/Le-TP-de-Keunotor3/optimisation/tp/tp2/LibOptim.py:35: RuntimeWarning: invalid value encountered in double_scalars
  alpha.append(rhoi*np.dot(si,r))
/Users/julienblanchon/Git/julien-blanchon/Le-TP-de-Keunotor3/optimisation/tp/tp2/LibOptim.py:38: RuntimeWarning: invalid value encountered in double_scalars
  r = r*(np.dot(sk, yk)/np.dot(yk, yk))


In [9]:
r

{'x_list': [array([-2., -2.]),
  array([-2.00012207, -2.00012207]),
  array([-2.00011782, -2.0002277 ]),
  array([-2.00026531, -2.00033869]),
  array([-2.0003755 , -2.00049965]),
  array([-2.0003755 , -2.00049965]),
  array([nan, nan]),
  array([nan, nan])],
 'cost_list': [array([2.83281062, 2.83281062]),
  array([2.83299244, 2.83325796]),
  array([2.83355071, 2.83372805]),
  array([2.83410972, 2.83440979]),
  array([2.83410972, 2.83440979]),
  array([nan, nan]),
  array([nan, nan])],
 'grad_list': [3.0240788312284432,
  3.0242706872545706,
  3.024288814976173,
  3.0245123401739584,
  3.0246969139940965,
  3.0246969139940965,
  nan],
 'step_list': [6.103515625e-05,
  6.103515625e-05,
  6.103515625e-05,
  6.103515625e-05,
  4.81482486096809e-35,
  4.81482486096809e-35,
  4.81482486096809e-35],
 'angle_list': [0.5485304671885517,
  0.8425915142414898,
  0.5403023058681395,
  0.5836937224051124,
  0.6513291448813762,
  nan,
  nan]}